In [1]:
%env TOKENIZERS_PARALLELISM=false
!pip install bertopic

env: TOKENIZERS_PARALLELISM=false
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 35.2 MB/s eta 0:00:00
  Installing build dependencies ... - \ | / - \ | done
  Getting requirements to build wheel ... - done
  Preparing metadata (pyproject.toml) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for hdbscan: filename=hdbscan-0.8.33-cp310-cp310-linux_x86_64.whl size=819467 sha256=b66dd3e8b0988cd196665b1f82c26ba9ade21753e2eecdeb7508da0432d6b86b
  Stored in directory: /root/.cache/pip/wheels/75/0b/3b/dc4f60b7cc455efaefb62883a7483e76f09d06ca81cf87d610
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=56c8413a0f41073171319e097d96ddecfbc9ab52302aba18388930f10d3122ce
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/

In [2]:
import pandas as pd
df = pd.read_csv(filepath_or_buffer='/kaggle/input/stranger-things-dialogue-dataset/stranger_things_all_dialogue.csv')
df['text'] = df['raw_text'] + ' ' + df['stage_direction'] + ' ' + df['dialogue'].fillna(value='')
df.head()

,season,episode,line,raw_text,stage_direction,dialogue,start_time,end_time,text
0,1,1,1,[crickets chirping],[crickets chirping],NaN,00:00:07,00:00:09,[crickets chirping] [crickets chirping]
1,1,1,2,[alarm blaring],[alarm blaring],NaN,00:00:49,00:00:51,[alarm blaring] [alarm blaring]
2,1,1,3,[panting],[panting],NaN,00:00:52,00:00:54,[panting] [panting]
3,1,1,4,[elevator descending],[elevator descending],NaN,00:01:01,00:01:02,[elevator descending] [elevator descending]
4,1,1,5,[elevator dings],[elevator dings],NaN,00:01:09,00:01:10,[elevator dings] [elevator dings]


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32519 entries, 0 to 32518
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   season           32519 non-null  int64 
 1   episode          32519 non-null  int64 
 2   line             32519 non-null  int64 
 3   raw_text         32519 non-null  object
 4   stage_direction  10678 non-null  object
 5   dialogue         26435 non-null  object
 6   start_time       32519 non-null  object
 7   end_time         32519 non-null  object
 8   text             10678 non-null  object
dtypes: int64(3), object(6)
memory usage: 2.2+ MB


In [4]:
from collections import Counter
text = ' '.join(df['text'].dropna().values).replace('[', '').replace(']', '')
counts = Counter(text.split())
print(counts.most_common(n=20))
unique = set(text.split())
print(len(unique))

[('music', 1801), ('playing', 1392), ('I', 960), ('you', 782), ('in', 769), ('sighs', 764), ('grunts', 637), ('the', 594), ('chuckles', 548), ('a', 522), ('to', 510), ('continues', 478), ('panting', 477), ('Russian', 476), ('Dustin', 458), ('Hopper', 418), ('Mike', 408), ('on', 377), ('Joyce', 368), ('is', 345)]
7127


In [5]:
from collections import defaultdict
document_counts = defaultdict(int)
for index, row in df.dropna(subset=['text']).iterrows():
    current = set(row['text'].replace('[', '').replace(']', '').split())
    for token in current:
        document_counts[token] += 1
percentages = {key: value/len(df['text'].dropna()) for key, value in document_counts.items()}
print(list(percentages.items())[:10])

[('crickets', 0.00028095148904289194), ('chirping', 0.001311106948866829), ('blaring', 0.002341262408690766), ('alarm', 0.003652369357557595), ('panting', 0.023038022101517137), ('descending', 0.00018730099269526128), ('elevator', 0.001311106948866829), ('dings', 0.0014047574452144597), ('breathing', 0.013017418992320659), ('heavily', 0.009833302116501217)]


In [6]:
pcts = sorted(list(percentages.items()), reverse=True, key=lambda x: x[1])
print(pcts[:10])

[('music', 0.08540925266903915), ('playing', 0.06518074545795093), ('I', 0.043079228319910096), ('sighs', 0.038209402509833304), ('in', 0.03568083910844728), ('you', 0.03511893613036149), ('grunts', 0.03137291627645627), ('chuckles', 0.0278141974152463), ('the', 0.02612848848098895), ('Dustin', 0.025285634013860272)]


In [7]:
from arrow import now
from bertopic import BERTopic
from plotly.express import histogram
from sklearn.feature_extraction.text import CountVectorizer

SAMPLE_FRACTION = 1.0
time_start = now()
large_df = df[['text']].sample(frac=SAMPLE_FRACTION, random_state=2023)
large_model = BERTopic(verbose=True, top_n_words=20, nr_topics=None, language='english', 
                       vectorizer_model=CountVectorizer(stop_words='english', lowercase=True,
                                                       min_df = 3),
                      embedding_model='all-MiniLM-L6-v2', )
large_topics, large_probs = large_model.fit_transform(large_df['text'].dropna().values)
print('After {} our model says we have {} topics.'.format(now() - time_start, len(set(large_topics))))
print('And we have {} documents with no topic ({}%).'.format(large_topics.count(-1), round(100 * large_topics.count(-1)/len(large_topics))))
histogram(x=[item for item in large_topics if item > -1], title = 'topic > -1 histogram')

/opt/conda/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/opt/conda/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/opt/conda/lib/python3.10/site-packages/umap/distances.py:1086: NumbaDeprecationWarning: The 'nopython' keyword argume

Batches:   0%|          | 0/334 [00:00<?, ?it/s]

2023-10-24 19:58:56,331 - BERTopic - Transformed documents to Embeddings
2023-10-24 19:59:26,854 - BERTopic - Reduced dimensionality
2023-10-24 19:59:27,233 - BERTopic - Clustered reduced embeddings


After 0:01:18.113836 our model says we have 291 topics.
And we have 1192 documents with no topic (11%).


In [8]:
large_model.visualize_topics(width=1000, height=1000)

In [9]:
from math import log10
from plotly.express import scatter

def visualize(model, term):
    word = []
    topic_probability = []
    word_weight = []
    overall_weight = []
    topics, probabilities = model.find_topics(search_term=term, top_n=20)
    for index, topic in enumerate(topics):
        probability = probabilities[index]
        topic_result = model.get_topic(topic=topic)
        for word_result in topic_result:
            word.append(word_result[0])
            topic_probability.append(probability)
            word_weight.append(word_result[1])
            overall_weight.append(-log10(probability * word_result[1]))
    scatter(data_frame=pd.DataFrame(data={'word': word, 'topic_probability': topic_probability, 'word_weight': word_weight,
                                         'overall_weight': overall_weight}),
            x='topic_probability', y='word_weight', text='word', color='overall_weight', log_x=True, log_y=True,
            color_continuous_scale='Reds_r', title=term, height=1500, range_y=[0.002, 1]
           ).update_traces(textposition='middle right').update_traces(marker_coloraxis=None).show()

visualize(model=large_model, term='grunt')

In [10]:
visualize(model=large_model, term='hopper')

In [11]:
visualize(model=large_model, term='chuckle')

In [12]:
from math import log10
from plotly.express import scatter_polar

def scale_theta(arg: list) -> list:
    min_value = min(arg)
    max_value = max(arg)
    return [360 * (item - min_value)/max_value for item in arg]

def scale_r(arg: list) -> list:
    return [1.5 + item for item in arg]


def visualize2(model, term):
    word = []
    topic_probability = []
    word_weight = []
    overall_weight = []
    topic_indices = []
    word_indices = []
    topics, probabilities = model.find_topics(search_term=term, top_n=20)
    for index, topic in enumerate(topics):
        probability = probabilities[index]
        topic_result = model.get_topic(topic=topic)
        for word_index, word_result in enumerate(topic_result):
            topic_indices.append(index)
            word.append(word_result[0])
            topic_probability.append(probability)
            word_weight.append(word_result[1])
            word_indices.append(word_index)
            overall_weight.append(log10(probability * word_result[1]))
            
    word_weight = scale_theta(arg=word_weight)
    word_indices = scale_theta(arg=word_indices)
    topic_indices = scale_r(arg=topic_indices)
            
    plot_df = pd.DataFrame(data={'word': word, 'word_indices': word_indices,
                                         'weight': overall_weight, 'topic_indices': topic_indices,
                                })
    plot_df = plot_df.sort_values(ascending=False, by='weight').drop_duplicates(subset=['word'])
    scatter_polar(data_frame=plot_df,
            r='topic_indices', theta='word_indices', text='word', color='weight',  
            color_continuous_scale='ice', title=term, height=1500,
           ).update_traces(textposition='middle right').show()
    return plot_df

t_df = visualize2(model=large_model, term='grunt')


In [13]:
t_df.sort_values(by='topic_indices', ascending=True).head()

,word,word_indices,weight,topic_indices
15,say,284.210526,-1.903208,1.5
17,wrong,322.105263,-1.976702,1.5
16,clicking,303.157895,-1.913375,1.5
1,wait,18.947368,-1.293800,1.5
14,thing,265.263158,-1.870508,1.5
